# [Advent of Code 2018](https://adventofcode.com/2018)

A collection of common functions to be used across problems, will add more as a generic use case for each comes up!

In [45]:
import os
import re
from collections import defaultdict, namedtuple
from itertools import cycle


Point = namedtuple('Point', 'x,y')


def Input(day):
    """Fetch the data input from disk."""
    filename = os.path.join('../data/advent2018/input{}.txt'.format(day))
    return open(filename)


def hamming_distance(s1, s2):
    """Number of non equal characters between two strings."""
    assert len(s1) == len(s2), 'Strings are not equal length'
    return sum(
        char1 != char2
        for char1, char2
        in zip(s1, s2)
    )

## [Day 1: Chronal Calibration](https://adventofcode.com/2018/day/1)

The first part simply requires us to sum the values in a list to apply all the changes from the given delas.

In [2]:
def parse_input(initial_data):
    res = []
    for data in initial_data.readlines():
        res.append(int(data))
    return res

data = parse_input(Input(1))

In [3]:
sum(data)

525

Second portion requires us to find the first repeated value if we continually sum items in the delta list. A set is ideal here as it's O(1) for membership tests.

In [4]:
def find_first_repeat(deltas):
    seen = set([0])
    position = 0
    
    for delta in cycle(deltas):
        position += delta
        if position in seen:
            break
        seen.add(position)
    return position

assert find_first_repeat([7, 7, -2, -7, -4]) == 14

In [5]:
find_first_repeat(data)

75749

# [Day 2: Inventory Management System](https://adventofcode.com/2018/day/2)

In [69]:
data = [line.strip() for line in Input(2).readlines()]

In [70]:
from collections import Counter

def calculate_checksum(data):
    num_threes = 0
    num_twos = 0
    for line in data:
        counts = Counter(line)
        if 3 in counts.values():
            num_threes += 1
        if 2 in counts.values():
            num_twos += 1
        
    return num_threes * num_twos


test_data = [
    "abcdef",
    "bababc",
    "abbcde",
    "abcccd",
    "aabcdd",
    "abcdee",
    "ababab",
]
assert calculate_checksum(test_data) == 12
calculate_checksum(data)
    

6944

We're told that we need to find two strings that have only one character difference between them, this is also known as the the [Hamming Distance](https://en.wikipedia.org/wiki/Hamming_distance). So we're looking for two strings for which the hamming distance bewteen them is one, simple enough!

In [77]:
def find_boxes(data):
    for index, row in enumerate(data[:-1]):
        for comparison in data[index:]:
            if hamming_distance(row, comparison) == 1:
                same_chars = [
                    char1
                    for (char1, char2) in zip(row, comparison)
                    if char1 == char2
                ]
                return ''.join(same_chars)

find_boxes(data)

'srijafjzloguvlntqmphenbkd'

## [Day 3: No Matter How You Slice It](https://adventofcode.com/2018/day/3)

In [48]:
def parse_input(data):
    reg = re.compile('#(\d+) @ (\d+),(\d+): (\d+)x(\d+)')
    datum = namedtuple('datum', 'claim,left,top,width,height')
    parsed = []
    for line in data:
        match = reg.match(line)
        
        parsed.append(
            datum(*[int(i) for i in match.groups()])
        )
    return parsed


def find_overlaps(data):
    overlaps = defaultdict(list)
    repeated = 0
    
    for datum in data:
        for x in range(datum.width):
            for y in range(datum.height):
                p = Point(datum.left + x, datum.top + y)
                overlaps[p].append(datum.claim)
    return overlaps


def count_overlaps(data):
    overlaps = find_overlaps(data)
    return len([
        claims for claims in overlaps.values()
        if len(claims) > 1
    ])
    
        
test_data = [
    '#1 @ 1,3: 4x4',
    '#2 @ 3,1: 4x4',
    '#3 @ 5,5: 2x2',
]
test_data = parse_input(test_data)
assert count_overlaps(test_data) == 4

data = parse_input(Input(3).readlines())
count_overlaps(data)

110827

In [53]:
def find_unique(data):
    remainin_claims = set(
        datum.claim for datum in data
    )
    overlaps = find_overlaps(data)
    
    for claims in overlaps.values():
        if len(claims) == 1:
            continue
        remainin_claims -= set(claims)
    assert len(list(remainin_claims)) == 1
    return list(remainin_claims)[0]


assert find_unique(test_data) == 3
find_unique(data)

116